<a href="https://colab.research.google.com/github/qonca/traditional_ml/blob/main/nlp_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name = 'imdb_reviews',
    split = ['train[:90%]', 'train[90%:]', 'test'],
    as_supervised = True
)

tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed = 42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.91JIYN_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.91JIYN_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.91JIYN_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode('utf-8')[:200], "...")
    print('Label:', label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


In [ ]:
vocab_size = 1000

text_vec_layer = tf.keras.layers.TextVectorization(max_tokens = vocab_size,
                                                   split = 'whitespace',
                                                   standardize = 'lower_and_strip_punctuation')
text_vec_layer.adapt(train_set.map(lambda review, label: review))

In [ ]:
text_vec_layer.get_vocabulary()[-50:]

[np.str_('jane'),
 np.str_('brothers'),
 np.str_('battle'),
 np.str_('apart'),
 np.str_('mess'),
 np.str_('development'),
 np.str_('casting'),
 np.str_('potential'),
 np.str_('20'),
 np.str_('open'),
 np.str_('effect'),
 np.str_('christmas'),
 np.str_('expecting'),
 np.str_('dream'),
 np.str_('outside'),
 np.str_('remake'),
 np.str_('manages'),
 np.str_('forward'),
 np.str_('bill'),
 np.str_('twist'),
 np.str_('attempts'),
 np.str_('deserves'),
 np.str_('create'),
 np.str_('70s'),
 np.str_('unlike'),
 np.str_('imdb'),
 np.str_('dumb'),
 np.str_('powerful'),
 np.str_('portrayed'),
 np.str_('setting'),
 np.str_('missing'),
 np.str_('fire'),
 np.str_('fairly'),
 np.str_('scifi'),
 np.str_('inside'),
 np.str_('background'),
 np.str_('married'),
 np.str_('business'),
 np.str_('ben'),
 np.str_('mark'),
 np.str_('recently'),
 np.str_('fantasy'),
 np.str_('air'),
 np.str_('pay'),
 np.str_('gay'),
 np.str_('present'),
 np.str_('monster'),
 np.str_('fighting'),
 np.str_('joke'),
 np.str_('plain'

In [ ]:
text_vec_layer(['it was a great movie'])

<tf.Tensor: shape=(1, 5), dtype=int64, numpy=array([[ 9, 14,  4, 86, 18]])>

In [ ]:
embed_layer = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = 16)
embed_layer(text_vec_layer(['it was a great movie']))

<tf.Tensor: shape=(1, 5, 16), dtype=float32, numpy=
array([[[-0.02475681,  0.01053506,  0.03605518, -0.02717016,
          0.03193918,  0.01953416, -0.01715529, -0.04037786,
          0.03136149,  0.04389728, -0.02657359,  0.00355778,
          0.02692816,  0.03121689, -0.00587882, -0.02698619],
        [ 0.04161317, -0.02925096, -0.02395902, -0.04541147,
         -0.01646777, -0.02860636, -0.00158725,  0.01344896,
          0.00618849,  0.03253598, -0.04028028, -0.00362792,
          0.0318573 , -0.01290613,  0.00913789,  0.02422526],
        [-0.0312584 ,  0.03772912,  0.02688647, -0.03894626,
         -0.03000141, -0.03602554,  0.03226919,  0.02929899,
          0.04258719, -0.04664918, -0.01415608, -0.03271428,
          0.02651367,  0.02170182, -0.02654914,  0.00035242],
        [ 0.0408602 ,  0.02056602,  0.00121849,  0.01785744,
         -0.01775251,  0.02642645, -0.00637827, -0.01238124,
         -0.00459417,  0.01195241, -0.03678205,  0.02887013,
         -0.02504517,  0.03224

In [ ]:
embed_size = 128

model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embed_size, mask_zero = True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Nadam(learning_rate = 1e-3),
    metrics = ['accuracy']
)
model.fit(train_set, validation_data = valid_set, epochs = 2)

Epoch 1/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 31s 39ms/step - accuracy: 0.6597 - loss: 0.5997 - val_accuracy: 0.8468 - val_loss: 0.3569
Epoch 2/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.8590 - loss: 0.3280 - val_accuracy: 0.8688 - val_loss: 0.3022


In [ ]:
embedding_weights = model.layers[1].get_weights()[0]

In [ ]:
import numpy as np
np.savetxt('embeddings.tsv', embedding_weights, delimiter = '\t')

In [ ]:
vocab = text_vec_layer.get_vocabulary()

with open('metafata.tsv', 'w', encoding = 'utf-8') as f:
    for word in vocab:
        word = word if word.strip() != '' else ' <PAD>'
        f.write(f"{word}\n")

#Generating Shakespearean Text Using a Character RNN

In [ ]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [ ]:
''.join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character',
                                                   standardize = 'lower')
text_vec_layer.adapt(shakespeare_text)
encoded = text_vec_layer([shakespeare_text][0])

In [ ]:
encoded -=2
vocab_size = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [ ]:
vocab_size

39

In [ ]:
dataset_size

1115394

In [ ]:
def to_dataset(sequence, length, seeed = None, shuffle = False, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length+1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_s: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed = seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:,:-1], window[:,1:])).prefetch(1)


In [ ]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length = length, shuffle = True,
                       seed = 42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length = length)
test_set = to_dataset(encoded[1_060_000:], length = length)

TypeError: to_dataset() got an unexpected keyword argument 'seed'

In [ ]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = 16),
    tf.keras.layers.GRU(128, return_sequences = True),
    tf.keras.layers.Dense(vocab_size, activation = 'softmax')
])

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Nadam(),
    metrics = ['accuracy']

)

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    'my_shakespeare_model.keras', monitor = 'val_accuracy', save_best_only = True

)

history = model.fit(train_set,
                    validation_data = valid_set,
                    epochs = 1,
                    callbacks = [model_ckpt])

ValueError: Input 0 of layer "gru_3" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 16)

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [ ]:
y_proba = shakespeare_model.predict(tf.constant(['To be or not to b']))[0,-1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]